# Over Under Predictor

## Data Cleaning Notebook

In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import random
import seaborn as sns
import math
import requests
%matplotlib inline
pd.set_option('display.max_columns', 100)

#### Import Gambling Statistics from ...

In [2]:
df = pd.read_csv('data/spreadspoke_scores.csv')

# General Cleaning

#### Drop all rows not contains an Over/Under Value

In [3]:
df = df.dropna(subset=['over_under_line'])

#### Convert the "Game Spread" to an Positive number 

In [4]:
df["spread_favorite"] = abs(df['spread_favorite'])

#### Add "Total" column for the Total Amount of Points Scored in a given game. Using Away Points Score  + Home Points Score

In [5]:
df['total'] = df['score_home'] + df['score_away'] 
# Change to an Interger
df = df.astype({'total': 'int'})

#### Drop all Rows without an Over/Under Line

In [6]:
df = df[df.over_under_line != ' ']
# Change to a Float
df = df.astype({'over_under_line': 'float'})

#### Create the Target Variable, Over/Under Result Column
* ##### 'Over' if the Total is greater than the Over/Under Line 
* ##### 'Under' if the Total is less than the Over/Under Line

In [7]:
df['over_under_result'] = np.where(df['total'] > df['over_under_line'], 'Over', 0)
df['over_under_result'] = np.where(df['total'] == df['over_under_line'], np.nan, df['over_under_result'])
df['over_under_result'] = np.where(df['total'] < df['over_under_line'], 'Under', df['over_under_result'])
# Drop all 'Pushed/Tied' Totals
df = df.dropna(subset=['over_under_result'])


In [8]:
df.shape

(10052, 19)

#### Update 'Washington Redskins' team name, to 'Washington Football team'

In [9]:
df['team_home'] = np.where(df['team_home'] == 'Washington Redskins', 'Washington Football Team', df['team_home'])
df['team_away'] = np.where(df['team_away'] == 'Washington Redskins', 'Washington Football Team', df['team_away'])

#### Correct all Alternative Names of Stadiums

In [10]:
df['stadium_name'] = df['stadium']
df = df.drop(columns=['stadium'])

# Create List of Stadium Names
stadium_list = list(df['stadium_name'])

# Create List of Hard Rock Stadium alternative stadium names
hard_rock_stadium_alternates = ['Joe Robbie Stadium','Pro Player Stadium','Dolphin Stadium']

# Create for loop that Changes all alternative name to 'Hard Rock Stadium'
for i, stadium in enumerate(stadium_list):
    if stadium in hard_rock_stadium_alternates:
        stadium_list[i] = 'Hard Rock Stadium'
        
# Update Column Row with new stadium list with Hard Rock Stadium instead of Alternatives
df['stadium_name'] = stadium_list

# Change other independent events of Stadium name changes
df['stadium_name'] = np.where(df['stadium_name'] == 'Tampa Stadium', 'Raymond James Stadium',df['stadium_name'])
df['stadium_name'] = np.where(df['stadium_name'] == 'Alltel Stadium', 'TIAA Bank Field',df['stadium_name'])
df['stadium_name'] = np.where(df['stadium_name'] == 'Jack Murphy Stadium', 'Qualcomm Stadium',df['stadium_name'])

#### Important Stadium Data from .....

In [11]:
stadium_df = pd.read_csv('data/nfl_stadiums.csv')

# Merge Stadium Data to the Final DataFrame
df = df.merge(stadium_df,on='stadium_name',how='left')

#Drop Uneeded Columns
df = df.drop(columns=['stadium_close','NAME'])

#### Get Stadium Locations

In [12]:
# Manually Add Stadium Locations
df['stadium_location'] = np.where(df['stadium_name'] == 'FedEx Field', 'Landover, MD',df['stadium_location'])
df['stadium_location'] = np.where(df['stadium_name'] == 'TIAA Bank Field', 'Jacksonville, FL',df['stadium_location'])

# Drop all Nulls
df = df.dropna(subset=['stadium_location'])

#### Drop Games Played before 1978

In [13]:
df = df[df.schedule_season > 1978]

#### Correct the Data the Stadium Opened

In [14]:
df[df['stadium_open'].isna()].stadium_name.value_counts()

FedEx Field         187
TIAA Bank Field      17
Rogers Centre         6
Rose Bowl             4
Alamo Dome            3
Stanford Stadium      1
Name: stadium_name, dtype: int64

In [15]:
# Manually change Open Years
df['stadium_open'] = np.where(df['stadium_name'] == 'FedEx Field', 1997,df['stadium_open'])
df['stadium_open'] = np.where(df['stadium_name'] == 'TIAA Bank Field', 1994,df['stadium_open'])
df['stadium_open'] = np.where(df['stadium_name'] == 'Rose Bowl', 1921,df['stadium_open'])
df['stadium_open'] = np.where(df['stadium_name'] == 'Alamo Dome', 1993,df['stadium_open'])
df['stadium_open'] = np.where(df['stadium_name'] == 'Stanford Stadium', 1921,df['stadium_open'])

df = df[df.stadium_name != 'Rogers Centre']

# Change to Integer
df = df.astype({'stadium_open': 'int'})

#### Fix the Type of Stadium

In [16]:
df['stadium_type'] = np.where(df['stadium_name'] == 'FedEx Field', 'outdoor',df['stadium_type'])
df['stadium_type'] = np.where(df['stadium_name'] == 'TIAA Bank Field', 'outdoor',df['stadium_type'])
df['stadium_type'] = np.where(df['stadium_name'] == 'Stanford Stadium', 'outdoor',df['stadium_type'])

#### Get the Zip Codes for all Stadiums

In [17]:
# Get Zipcodes by splitting last 5 elements from Address Values
df['zipcode'] = df['stadium_address'].str[-5:]

# Adding Zip Codes to Stadiums Without Address
df['zipcode'] = np.where(df['stadium_name'] == 'Rose Bowl', 91103,df['zipcode'])
df['zipcode'] = np.where(df['stadium_name'] == 'Stanford Stadium', 94305,df['zipcode'])
df['zipcode'] = np.where(df['stadium_name'] == 'FedEx Field', 20785,df['zipcode'])
df['zipcode'] = np.where(df['stadium_name'] == 'TIAA Bank Field', 32202,df['zipcode'])
df['zipcode'] = np.where(df['stadium_name'] == 'Mercedes-Benz Stadium', 30313,df['zipcode'])
df['zipcode'] = np.where(df['stadium_name'] == 'SoFi Stadium', 90301,df['zipcode'])
df['zipcode'] = np.where(df['stadium_name'] == 'Allegiant Stadium', 89118,df['zipcode'])

# Drop Stadiums outside the U.S.
df = df[df.stadium_name != 'Wembley Stadium']

#### Fix the Surface Type for every Stadium

In [18]:
# Change other names for Turf and Grass
df['stadium_surface'] = np.where(df['stadium_surface'] == 'Hellas Matrix Turf', 'FieldTurf',df['stadium_surface'])
df['stadium_surface'] = np.where(df['stadium_surface'] == 'Grass, Turf (1971-1974)', 'Grass',df['stadium_surface'])

# Make corresponding list from the stadium name & stadium surface columns
stadium_list = list(df['stadium_name'])
surface_list = list(df['stadium_surface'])
# Create list for All Stadiums with Turf Surfaces
turf_stadiums = ['Giants Stadium','Texas Stadium','Hubert H. Humphrey Metrodome','RCA Dome','Veterans Stadium',
                'Foxboro Stadium','Pontiac Silverdome','Three Rivers Stadium','Edward Jones Dome','Cinergy Field',
                'Seattle Kingdome','Houston Astrodome','Busch Memorial Stadium','Mall of America Field',
                'Husky Stadium',]
# Create for loops to add Correct Surface for every Stadium in turf_stadium list
for i, stadium in enumerate(stadium_list):
    # Get the Index for the Stadium if the Stadium is in Turf Stadium List
    if stadium in turf_stadiums:
        surface_list[i] = 'FieldTurf'
    # Use the index to change the corresponding Surface Type
df['stadium_surface'] = surface_list

# Change all the Stadiums that dont have Turf to Grass values
df['stadium_surface'] = np.where(df['stadium_surface'] == 'FieldTurf', 'FieldTurf', 'Grass')

#### Import Stadium Capacities

In [19]:
# Manually Import Each Stadium Capacity

df['stadium_capacity'] = np.where(df['stadium_name'] == 'Giants Stadium',80242,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Candlestick Park',69732,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Sun Life Stadium',64767,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Texas Stadium',65675,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Hubert H. Humphrey Metrodome', 64121,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'RCA Dome',60567,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Veterans Stadium',65352,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'FedEx Field', 82000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Foxboro Stadium',60292,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Pontiac Silverdome',80311,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Mile High Stadium', 75000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Three Rivers Stadium',59000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Edward Jones Dome', 67277,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Cinergy Field', 59754,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Seattle Kingdome',66000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == "Houlihan's Stadium", 50000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Houston Astrodome', 65000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'RFK Memorial Stadium',45596,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Cleveland Municipal Stadium', 81000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Anaheim Stadium', 69008,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Atlanta-Fulton County Stadium', 60606,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Busch Memorial Stadium', 60000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Orange Bowl', 75000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Memorial Stadium (Baltimore)', 50000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Sun Devil Stadium', 53599,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Mall of America Field', 64121,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Metropolitan Stadium', 41200,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Wembley Stadium', 86000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Husky Stadium', 70000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'TCF Bank Stadium', 50805,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'TIAA Bank Field', 67814,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == "Memorial Stadium (Champaign)", 60670,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == "Memorial Stadium (Clemson)", 74000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Liberty Bowl Memorial Stadium', 58325,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Vanderbilt Stadium', 40550,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Rose Bowl', 92542,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == "Tiger Stadium (LSU)", 100000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Tulane Stadium', 70000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Stanford Stadium', 50000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Rice Stadium', 47000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Tulane Stadium', 70000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Stanford Stadium', 50000,df['stadium_capacity'])
df['stadium_capacity'] = np.where(df['stadium_name'] == 'Rice Stadium', 47000,df['stadium_capacity'])

# Take out comma's from the Stadium Capacity
df['stadium_capacity'] = df.stadium_capacity.replace(',','', regex=True)

df = df[df.stadium_name != 'Estadio Azteca']
df = df[df.stadium_name != 'Twickenham Stadium']

# Stadium Capacity to Integer
df = df.astype({'stadium_capacity': 'int'})

In [20]:
#df[df['stadium_capacity'].isna()].stadium_name.value_counts()

#### Complete Column for which Week of a Season a given game was played

In [21]:
# Week 1-17 Regular Season Games
# Week 18 and Above Playoff Games

# Wildcard Playoff Games set as Week 18
df['schedule_week'] = np.where(df['schedule_week'] == "Wildcard", 18,df['schedule_week'])
df['schedule_week'] = np.where(df['schedule_week'] == "WildCard", 18,df['schedule_week'])
# Divional Playoff Games set as Week 19
df['schedule_week'] = np.where(df['schedule_week'] == 'Division', 19,df['schedule_week'])
# Conference Championship Games set as Week 20
df['schedule_week'] = np.where(df['schedule_week'] == 'Conference', 20,df['schedule_week'])
# Superbowl Championship Games set as Week 21
df['schedule_week'] = np.where(df['schedule_week'] == 'Superbowl', 21,df['schedule_week'])
df['schedule_week'] = np.where(df['schedule_week'] == 'SuperBowl', 21,df['schedule_week'])

# Change Week Number to Integers 
df = df.astype({'schedule_week': 'int'})

#### Get the Day of the Week a game was played 

In [22]:
# converting the string to datetime format
import datetime
df['schedule_date'] = pd.to_datetime(df['schedule_date'])


# Get Day of the Week Game was Played
# 0 = Monday, 
# 1 = Tuesday
# 2 = Wednesday
# 3 = Thursday
# 4 = Friday
# 5 = Saturday
# 6 = Sunday
df['weekday'] = df['schedule_date'].dt.dayofweek

### Getting Distance Between Home & Away Team

#### Start with creating a column for Away Team Zipcodes

In [23]:
# To get an accurate Zipcode, get dataframe from early in the week & not a neutral stadium.
early = df[(df.schedule_week < 4)&(df.stadium_neutral == False)]

# Get a Dataframe of just the Teams and Their Corresponding Zipcodes
zip_df = early[['team_home', 'zipcode']]


# Change Column Names to use as Zipcode when Teams Away 
zip_df.columns = ['team_away','zipcode_away']

# Drop Duplicates so Teams only have one Zipcode per Season
zip_df = zip_df.drop_duplicates(subset = 'team_away')

## Merge Away Zip Codes to Original Dataset ##
df = df.merge(zip_df,how='left')


#### Create a Function to get distance between Two Zip Codes

In [24]:
## Install The Needed Packages ##
#!pip install uszipcode
#!pip install mpu

from uszipcode import SearchEngine
import mpu

# Zipcode Reader note Reading zipcode of Tempe,AZ so changed to Scottsdale,AZ
df['zipcode'] = np.where(df['zipcode'] == '85287','85054',df['zipcode'])
df['zipcode_away'] = np.where(df['zipcode_away'] == '85287','85054',df['zipcode_away'])

# Instantiate Zip Code Reader
search = SearchEngine(simple_zipcode=True)

# Define Function that Takes two Zip Codes and Returns Distance between the Zip Codes
def get_dist(zipcode_1,zipcode_2):
    # Get Zip Code 1 Latitude and Longitude
    zip1 = search.by_zipcode(zipcode_1)
    lat1 = zip1.lat
    long1 = zip1.lng
    # Get Zip Code 2 Latitude and Longitude
    zip2 = search.by_zipcode(zipcode_2)
    lat2 = zip2.lat
    long2 = zip2.lng
    # Return Distance
    return mpu.haversine_distance((lat1,long1),(lat2,long2))

# Create Column Taking in the Zip Code of the Home Team and Zip Code of the Away Team. 
# Returning a column returning the distance between the Two Zip Codes
df['dist_diff'] = df[['zipcode', 'zipcode_away']].apply(lambda x: get_dist(*x), axis = 1)

### Add Team Data from .....

#### Build Webscraper

In [25]:
# Initial Scrape Settin Up Team Statistics Dataframe
# URL to Pro Football Reference where there Data is located
url = 'https://www.pro-football-reference.com/years/'+'1978'+'/'
# Retrieve the First Dataframe containing AFC Conference Data
afc_df = pd.read_html(url)[0]
# Set the Year of the Season
afc_df['Season_Year'] = '1978'
# Retrieve the First Dataframe containing NFC Conference Data
nfc_df = pd.read_html(url)[1]
# Set the Year of the Season
nfc_df['Season_Year'] = '1978'
# Merge the Two Dataframes
df_1 = [nfc_df,afc_df]
team_df = pd.concat(df_1)
team_df = team_df.reset_index(drop=True)
# Delete Uneeded AFC and NFC Rows
team_df = team_df[~team_df.Tm.str.contains("AFC")]
team_df = team_df[~team_df.Tm.str.contains("NFC")]
# Delete Uneeded Marks on Team Names
team_df['Tm'] = team_df['Tm'].str.replace('+','')
team_df['Tm'] = team_df['Tm'].str.replace('*','')

In [26]:
# Get a list of the Season needed to iterate through URL
years = list(map(str,df.schedule_season.unique()))

# Set up loop to retrieve all data for Needed Years(Seasons)
for year in years:
    # Set up url with corrisponding Year(Season) for the destination of the data
    url = 'https://www.pro-football-reference.com/years/'+year+'/'
    # get afc dataframe and add year
    afc_df = pd.read_html(url)[0]
    afc_df['Season_Year'] = year
    # get nfc dataframe and add year
    nfc_df = pd.read_html(url)[1]
    nfc_df['Season_Year'] = year
    # Combine dataframes
    df_1 = [nfc_df,afc_df]
    df_1 = pd.concat(df_1)
    team_df = pd.merge(team_df, df_1,how = 'outer')
    # Clean Master Dataframe
    team_df = team_df.reset_index(drop=True)
    team_df = team_df[~team_df.Tm.str.contains("AFC")]
    team_df = team_df[~team_df.Tm.str.contains("NFC")]
    team_df['Tm'] = team_df['Tm'].str.replace('+','')
    team_df['Tm'] = team_df['Tm'].str.replace('*','')
    
    

#### Update NFL team names

In [27]:
team_df['Tm'] = np.where(team_df['Tm'] == 'Washington Redskins', 'Washington Football Team',team_df['Tm'])

#### Ready the Team Season data for Data Merge

In [28]:
# Convert Columns to Float
team_df[['W', 'L', 'T', 'W-L%', 'PF', 'PA', 'PD', 'MoV', 'SoS', 'SRS',
       'OSRS', 'DSRS', 'Season_Year']] = team_df[['W', 'L', 'T', 'W-L%', 'PF', 'PA', 'PD', 'MoV', 'SoS', 'SRS',
       'OSRS', 'DSRS', 'Season_Year']].astype(float)

# Seperate Data into Two Dataframe for Home Team & Away Team
home_team_df = team_df
away_team_df = team_df

# Add 1 to the Season Year because we are using previous season record
home_team_df['schedule_season'] = home_team_df['Season_Year'] + 1 
# No Value for Tied Games = 0
home_team_df['T'] = home_team_df['T'].fillna(0)
# Create a Points For per Games Column
home_team_df['PF_per_game'] = home_team_df['PF']/(home_team_df['W'] + home_team_df['L'] + home_team_df['T'])
# Create a Points Against per Games Column
home_team_df['PA_per_game'] = home_team_df['PA']/(home_team_df['W'] + home_team_df['L'] + home_team_df['T'])
# Drop Redundant Team Statistics
home_team_df = home_team_df.drop(columns=['Season_Year', 'W', 'L', 'T','PF','PA'])

# add 'home_' before all Home Team Columns
col_names = list(home_team_df.columns)
home_col_names = ["home_" + col for col in col_names]
home_team_df.columns = home_col_names
# Change 'home_tm' column name to corrispond with Final Dataset 'team_home'
home_team_df['team_home'] = home_team_df['home_Tm']
# Change 'home_schedule_season' column name to corrispond with Final Dataset 'schedule_season'
home_team_df['schedule_season'] = home_team_df['home_schedule_season']
# Drop Uneeded Columns
home_team_df = home_team_df.drop(columns=['home_Tm','home_schedule_season'])

# Left Merge Team Data to Final Dataframe. 
# Will merge on team_home and schedule_season
final_df = df.merge(home_team_df,how='left')

#### Reiterate the Same process for Away Team Data

In [29]:
away_team_df['schedule_season'] = away_team_df['Season_Year'] + 1
away_team_df['T'] = away_team_df['T'].fillna(0)
away_team_df['PF_per_game'] = away_team_df['PF']/(away_team_df['W'] + away_team_df['L'] + away_team_df['T'])
away_team_df['PA_per_game'] = away_team_df['PA']/(away_team_df['W'] + away_team_df['L'] + away_team_df['T'])
away_team_df = away_team_df.drop(columns=['Season_Year', 'W', 'L', 'T','PF','PA'])

col_names = list(away_team_df.columns)
away_col_names = ["away_" + col for col in col_names]
away_team_df.columns = away_col_names
away_team_df['team_away'] = away_team_df['away_Tm']
away_team_df['schedule_season'] = away_team_df['away_schedule_season']
away_team_df = away_team_df.drop(columns=['away_Tm','away_schedule_season'])

final_df = final_df.merge(away_team_df,how='left')

#### Drop All null values for team names that didnt match Final Dataframe

In [30]:
final_df = final_df.dropna(subset=['home_W-L%'])
final_df = final_df.dropna(subset=['away_W-L%'])

### Add Preseason Superbowl Odds

#### Build Webscraper to retrieve Preseason Superbowl Odds

In [31]:
# url with the destination of the Needed Data
url = "https://www.pro-football-reference.com/years/2020/preseason_odds.htm#preseason_odds::none"
# Set up the Orginial Dataframe with 2020 Preseason Superbowl Odds
superbowl_odds = pd.read_html(url)[0]
superbowl_odds['Season_Year'] = '2020'

In [32]:
# Use the years list to determine the data retrieved 
for year in years:
    url = "https://www.pro-football-reference.com/years/"+year+"/preseason_odds.htm"
    superbowl_odds_1 = pd.read_html(url)[0]
    superbowl_odds_1['Season_Year'] = year
    # Merge Superbowl Odds to original Superbowl Odds dataset
    superbowl_odds = pd.merge(superbowl_odds, superbowl_odds_1,how = 'outer')

# Drop Uneeded Columns
superbowl_odds_df = superbowl_odds.drop(columns=['W/L O-U','Record'])

#### Fix outdated team names

In [33]:
superbowl_odds_df['Tm'] = np.where(superbowl_odds_df['Tm'] == 'Washington Redskins', 'Washington Football Team',superbowl_odds_df['Tm'])

#### Seperate dataframe into two matching Away Team and Home Team dataframs

In [34]:
# Change schedule_season to an integer to match dataframes
superbowl_odds_df[['Season_Year']] = superbowl_odds_df[['Season_Year']].astype(float)

# Split to Home and Away Teams
home_superbowl_odds_df = superbowl_odds_df
away_superbowl_odds_df = superbowl_odds_df

#### Prepare Superbowl Data to Merge into Final Dataframe

In [35]:
# Change home team column names to match Final Dataframe Column names
home_superbowl_odds_df.columns = ['team_home', 'home_superbowl_odds','schedule_season']
# Left Merge match team_home and 'schedule_season'
final_df = final_df.merge(home_superbowl_odds_df,how='left')

# Reiterate Process for the Away Team
away_superbowl_odds_df.columns = ['team_away', 'away_superbowl_odds','schedule_season']
final_df = final_df.merge(away_superbowl_odds_df,how='left')

# Will Amend if there is Time, For now drop 2020 data
final_df = final_df.dropna(subset=['home_superbowl_odds'])

#### Get Differences of Team Stats

In [36]:
# Gets differnce in Team Data Statistics
final_df['diff_W-L%'] = abs(final_df['home_W-L%'] - final_df['away_W-L%'])
final_df['diff_PD'] = abs(final_df['home_PD']- final_df['away_PD'])
final_df['diff_MoV'] = abs(final_df['home_MoV']- final_df['away_MoV'])
final_df['diff_SoS'] = abs(final_df['home_SoS']- final_df['away_SoS'])
final_df['diff_SRS'] = abs(final_df['home_SRS']- final_df['away_SRS'])
final_df['diff_OSRS'] = abs(final_df['home_OSRS']- final_df['away_OSRS'])
final_df['diff_PF_per_game'] = abs(final_df['home_PF_per_game']- final_df['away_PF_per_game'])
final_df['diff_PA_per_game'] = abs(final_df['home_PA_per_game']- final_df['away_PA_per_game'])
final_df['diff_superbowl_odds'] = abs(final_df['home_superbowl_odds']- final_df['away_superbowl_odds'])

# Get Longitudes and Latitudes

In [37]:
#!pip install pgeocode

In [38]:
import pgeocode

In [39]:
all_zips = list(final_df.zipcode.unique())
all_zips = [str(i) for i in all_zips]

In [40]:
nomi = pgeocode.Nominatim('us')
long_lat_df = nomi.query_postal_code(all_zips)

In [41]:
long_lat_df = long_lat_df.drop(columns =['country_code','place_name','state_name','state_code','county_name',
                               'county_code','community_name','community_code','accuracy'])
final_df = final_df.drop(columns =['LATITUDE','LONGITUDE'])

In [42]:
long_lat_df.columns = ['zipcode','LATITUDE','LONGITUDE']

In [43]:
final_df = pd.merge(final_df, long_lat_df,how='inner',on=["zipcode"])

In [44]:
final_df = final_df.sort_values(by='schedule_date').reset_index(drop=True)

In [45]:
final_df.to_csv('data/final_df.csv', index = False, header=True)

# Weather Data

In [46]:
def get_UNIX(x):
    unix = pd.to_datetime([x + ' 21:00:00']).astype(int) / 10**9
    return int(unix[0])

final_df['schedule_date'] = final_df['schedule_date'].dt.strftime('%Y-%m-%d')
final_df['unix'] = final_df['schedule_date'].apply(get_UNIX)

In [47]:
final_df['LATITUDE']=final_df['LATITUDE'].astype(str)
final_df['LONGITUDE']= final_df['LONGITUDE'].astype(str)
final_df['unix']= final_df['unix'].astype(str)

In [48]:
def get_weather(latitude,longitude,unix):
    url = "https://dark-sky.p.rapidapi.com/"+latitude+','+longitude+','+unix
    headers = {
    'x-rapidapi-key': "ace28e75e5msh0810e63ff3d1c2dp16c863jsnf40bb24e6796",
    'x-rapidapi-host': "dark-sky.p.rapidapi.com"}
    response = requests.request("GET", url, headers=headers)
    data = response.json()
    weather = data['currently']
    weather = pd.DataFrame(weather, index=[0])
    return weather

In [50]:
weather_df = get_weather('27.9625','-82.4895','305067600')
for index, row in first_thousand.iterrows():
    latitude = row['LATITUDE']
    longitude = row['LONGITUDE']
    unix = row['unix']
    weather_df2 = get_weather(latitude,longitude,unix)
    weather_df = pd.concat([weather_df, weather_df2])

NameError: name 'first_thousand' is not defined

weather_df = weather_df.reset_index().drop([0])

weather_df = weather_df.reset_index()
weather_df = weather_df.drop(columns = ['level_0','index'])

In [ ]:
print(weather_df.shape)
weather_df
#weather_df.to_csv('data/weather_df.csv', index = False, header=True)

In [ ]:
final_weather = weather_df

In [51]:
final_weather = pd.read_csv('data/weather_df.csv')

In [52]:
final_weather = final_weather.drop(columns = ['index','level_0'])

In [53]:
mergedDf = final_df.merge(final_weather, left_index=True, right_index=True)

In [54]:
mergedDf = mergedDf.drop(columns = ['precipAccumulation','ozone','windGust','weather_temperature',
                                   'weather_wind_mph','weather_humidity','weather_detail'])

In [55]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

mergedDf.isnull().sum()

schedule_date                      0
schedule_season                    0
schedule_week                      0
schedule_playoff                   0
team_home                          0
score_home                         0
score_away                         0
team_away                          0
team_favorite_id                   0
spread_favorite                    0
over_under_line                    0
stadium_neutral                    0
total                              0
over_under_result                  0
stadium_name                       0
stadium_location                   0
stadium_open                       0
stadium_type                       0
stadium_address                    0
stadium_weather_station_code       0
stadium_weather_type               0
stadium_capacity                   0
stadium_surface                    0
STATION                         2204
ELEVATION                       2204
zipcode                            0
weekday                            0
z

In [56]:
mergedDf.precipType.value_counts()

rain    1471
snow     181
Name: precipType, dtype: int64

In [57]:
mergedDf['summary'] = mergedDf.summary.fillna('None')
mergedDf['icon'] = mergedDf.icon.fillna('None')
mergedDf['precipIntensity'] = mergedDf.precipIntensity.fillna(0)
mergedDf['precipProbability'] = mergedDf.precipProbability.fillna(0)
mergedDf['precipType'] = mergedDf.precipProbability.fillna(0)
mergedDf['windBearing'] = mergedDf.windBearing.fillna(0)
mergedDf['visibility'] = mergedDf.visibility.fillna(0)


In [58]:
mergedDf.to_csv('data/final_df.csv', index = False, header=True)